In [1]:
import json
from authorization import SpotifyAuth

In [2]:
%load_ext autoreload

In [3]:
%autoreload 2

In [4]:
with open('client-secrets-rnr.json','r+') as secrets_file:
    secrets = json.load(secrets_file)

client_id = secrets["client_id"]
client_secret = secrets["client_secret"]
redirect_url = secrets["redirect_url"]

In [5]:
spotify_authorisation = SpotifyAuth(client_id=client_id, client_secret=client_secret, redirect_url=redirect_url)
spotify_authorisation.set_access_token_from_file('access_token.json')

# Get User Details

In [6]:
import spotify_functions as spotify

In [7]:
spotify.get_user_details(spotify_authorisation.get_access_token())

{'display_name': 'sdspot2034',
 'external_urls': {'spotify': 'https://open.spotify.com/user/sdspot2034'},
 'href': 'https://api.spotify.com/v1/users/sdspot2034',
 'id': 'sdspot2034',
 'images': [{'url': 'https://i.scdn.co/image/ab67757000003b8280153af8b79b3144b24f1291',
   'height': 64,
   'width': 64},
  {'url': 'https://i.scdn.co/image/ab6775700000ee8580153af8b79b3144b24f1291',
   'height': 300,
   'width': 300}],
 'type': 'user',
 'uri': 'spotify:user:sdspot2034',
 'followers': {'href': None, 'total': 9},
 'country': 'IN',
 'product': 'premium',
 'explicit_content': {'filter_enabled': False, 'filter_locked': False},
 'email': 'shreyandas2034@gmail.com'}

In [8]:
import datetime
yesterday = datetime.datetime.now() - datetime.timedelta(days=1)
# print(yesterday.timestamp()*1e3)
cdc_time = int(yesterday.timestamp()*1e3)

# Get All Songs Played in the last one day

In [9]:
token = spotify_authorisation.get_access_token()
history = spotify.get_listening_history(token, cdc_time)['items']

## EDA over recent history

### Number of songs played

In [10]:
len(history)

12

### Fetch details of a song

#### See available data for a song

In [11]:
history[0].keys()

dict_keys(['track', 'played_at', 'context'])

In [12]:
list(history[0]['track'].keys())

['album',
 'artists',
 'disc_number',
 'duration_ms',
 'explicit',
 'external_ids',
 'external_urls',
 'href',
 'id',
 'is_local',
 'name',
 'popularity',
 'preview_url',
 'track_number',
 'type',
 'uri']

In [13]:
history[0]['played_at']

'2024-08-04T06:59:20.791Z'

In [14]:
print(history[0]['context'])

{'type': 'playlist', 'external_urls': {'spotify': 'https://open.spotify.com/playlist/2m5BrqANdf443IYDvaYIRG?sessionId=1722729600003&mode=enhanced'}, 'href': 'https://api.spotify.com/v1/playlists/2m5BrqANdf443IYDvaYIRG', 'uri': 'spotify:playlist:2m5BrqANdf443IYDvaYIRG'}


# Transform

## Dimensions

### DIM_ARTIST

In [15]:
import pandas as pd
from sqlalchemy import Table, MetaData, Column, Integer, String
import sql_functions

In [16]:
# stacked list-comprehension
artist_list = [artist for artists in [play['track']['artists'] for play in history] for artist in artists]

In [17]:
df_artists = pd.DataFrame(artist_list)
df_artists = df_artists.drop(columns=['href','uri','external_urls'])
df_artists = df_artists.rename(columns={'id':'spotify_id','name':'artist_name','type':'artist_type'})
df_artists.drop_duplicates(inplace=True)
df_artists

,spotify_id,artist_name,artist_type
0,23zg3TcAtWQy7J6upgbUnj,USHER,artist
1,0TnOYISbd1XYRBk9myaseg,Pitbull,artist
2,0ekbDNE2eOq8QiaXM34wer,Howie Day,artist
3,5ILrArfIV0tMURcHJN8Q07,Vanessa Carlton,artist
4,6zvul52xwTWzilBZl6BUbT,Pixies,artist
5,1KCSPY1glIKqW2TotWuXOR,P!nk,artist
6,6fAEFVqFolcEhXeReay5u6,Loquat,artist
7,6deZN1bslXzeGvOLaLMOIF,Nickelback,artist
8,3AQRLZ9PuTAozP28Skbq8V,The Script,artist
9,2C9n4tQgNLhHPhSCmdsQnk,David Archuleta,artist


#### Load (Type 1 SCD)

In [18]:
metadata = MetaData()

# Define the dimension table schema
dim_table = Table('DIM_ARTIST', metadata,
    Column('artist_wid', Integer, primary_key=True, autoincrement=True),
    Column('spotify_id', String(255), nullable=False, unique=True),
    Column('artist_name', String(255)),
    Column('artist_type', String(255))
)

In [30]:
sql_functions.save_to_sql(df_artists, dim_table, key_cols=['spotify_id'], wid=['artist_wid'], mode='upsert')

### DIM_SONG

In [17]:
history[0]['track'].keys()

dict_keys(['album', 'artists', 'disc_number', 'duration_ms', 'explicit', 'external_ids', 'external_urls', 'href', 'id', 'is_local', 'name', 'popularity', 'preview_url', 'track_number', 'type', 'uri'])

In [18]:
history[0]['track']

{'album': {'album_type': 'single',
  'artists': [{'external_urls': {'spotify': 'https://open.spotify.com/artist/5pKCCKE2ajJHZ9KAiaK11H'},
    'href': 'https://api.spotify.com/v1/artists/5pKCCKE2ajJHZ9KAiaK11H',
    'id': '5pKCCKE2ajJHZ9KAiaK11H',
    'name': 'Rihanna',
    'type': 'artist',
    'uri': 'spotify:artist:5pKCCKE2ajJHZ9KAiaK11H'},
   {'external_urls': {'spotify': 'https://open.spotify.com/artist/5K4W6rqBFWDnAN6FQUkS6x'},
    'href': 'https://api.spotify.com/v1/artists/5K4W6rqBFWDnAN6FQUkS6x',
    'id': '5K4W6rqBFWDnAN6FQUkS6x',
    'name': 'Kanye West',
    'type': 'artist',
    'uri': 'spotify:artist:5K4W6rqBFWDnAN6FQUkS6x'},
   {'external_urls': {'spotify': 'https://open.spotify.com/artist/4STHEaNw4mPZ2tzheohgXB'},
    'href': 'https://api.spotify.com/v1/artists/4STHEaNw4mPZ2tzheohgXB',
    'id': '4STHEaNw4mPZ2tzheohgXB',
    'name': 'Paul McCartney',
    'type': 'artist',
    'uri': 'spotify:artist:4STHEaNw4mPZ2tzheohgXB'}],
  'external_urls': {'spotify': 'https://open.s